In [55]:
import pandas as pd
import numpy as np
import networkx as nx
import pickle
import matplotlib.pyplot as plt

In [91]:
with open('twi22/processed/topic_tweet1_covid.pkl', 'rb') as f:
    G = pickle.load(f)
tweet = pd.read_csv('twi22/processed/topic_tweet1_covid_score.csv')

# with open('twi22/processed/topic_tweet2_war.pkl', 'rb') as f:
#     G = pickle.load(f)
# tweet = pd.read_csv('twi22/processed/topic_tweet2_war_score.csv')

# with open('twi22/processed/topic_tweet3_climate.pkl', 'rb') as f:
#     G = pickle.load(f)
# tweet = pd.read_csv('twi22/processed/topic_tweet3_climate_score.csv')

tweet.set_index('uid', inplace=True)
print("Nodes: ", len(G.nodes()), "Len tweet: ", len(tweet))
print("Edges: ", len(G.edges()))

Nodes:  195491 Len tweet:  195491
Edges:  1341914


In [82]:
# with bot inf vs. without bot inf
from scipy import stats
opi_b, opi_o = [], []
for node in G.nodes:
    if G.nodes[node]['label']=='bot':
        continue
    following_nodes = list(G.successors(node))
    condition = []
    for check_node_id in following_nodes:
        if tweet.loc[check_node_id]['StScore']<-0.8:    # sentiment threshold
            if G.nodes[check_node_id]['label']=='bot':
                condition.append(1)
    exist_2 = 1 in set(condition)
    if exist_2:
        opi_b.append(tweet.loc[node]['StScore'])
    else:
        opi_o.append(tweet.loc[node]['StScore'])


mu_b, mu_o = np.mean(opi_b), np.mean(opi_o)
t_stats, p_value = stats.ttest_ind(opi_b, opi_o)
print("ave b: {:.4f}, ave o: {:.4f}, p val: {:.4f}".format(mu_b, mu_o, p_value))

ave b: 0.1749, ave o: 0.1892, p val: 0.5045


## Analyzing the Influencers

In [54]:

# i = 0
# bots_follower_sent = {'id': [], 'follower_sent': []}
# humans_follower_sent = {'id': [], 'follower_sent': []}

# for node in G.nodes:
#     if tweet.loc[node]['StScore']<-0.8: # negative
#         if G.nodes[node]['label'] == 'bot':
#             bots_follower_sent['id'].append(node)

#             followers_sent = []
#             followers_nodes = list(G.successors(node))
#             for follower in followers_nodes:
#                 if G.nodes[follower]['label']=='human':
#                     followers_sent.append(tweet.loc[follower]['StScore'])
#             bots_follower_sent['follower_sent'].append(np.mean(followers_sent))
#         if G.nodes[node]['label'] == 'human':
#             humans_follower_sent['id'].append(node)

#             followers_sent = []
#             followers_nodes = list(G.successors(node))
#             for follower in followers_nodes:
#                 if G.nodes[follower]['label']=='human':
#                     followers_sent.append(tweet.loc[follower]['StScore'])
#             humans_follower_sent['follower_sent'].append(np.mean(followers_sent))

# # bots
# # count bots' relations
# bots_follower_sent = pd.DataFrame(bots_follower_sent)
# # followers
# rel_dt1, rel_dt2 = pd.read_csv('edges/followers.csv'), pd.read_csv('edges/following.csv')
# rel_dt1.set_index('source_id', inplace=True)
# rel_dt2.set_index('target_id', inplace=True)
# bots_follower_sent['followers'] = ''
# val_c1, val_c2 = rel_dt1.index.value_counts(), rel_dt2.index.value_counts()
# for i in range(len(bots_follower_sent)):
#         bots_follower_sent.at[i, 'followers'] = val_c1.get(bots_follower_sent['id'][i], 0) + val_c2.get(bots_follower_sent['id'][i], 0)
# # following
# rel_dt1, rel_dt2 = pd.read_csv('edges/following.csv'), pd.read_csv('edges/followers.csv')
# rel_dt1.set_index('source_id', inplace=True)
# rel_dt2.set_index('target_id', inplace=True)
# bots_follower_sent['following'] = ''
# val_c1, val_c2 = rel_dt1.index.value_counts(), rel_dt2.index.value_counts()
# for i in range(len(bots_follower_sent)):
#         bots_follower_sent.at[i, 'following'] = val_c1.get(bots_follower_sent['id'][i], 0) + val_c2.get(bots_follower_sent['id'][i], 0)
# # membership, mentioned
# rel1 = ['membership', 'mentioned']
# for rel in rel1:
#         rel_dt = pd.read_csv('edges/'+rel+'.csv')
#         rel_dt.set_index('target_id', inplace=True)
#         bots_follower_sent[rel] = ''
#         val_c = rel_dt.index.value_counts()
#         for i in range(len(bots_follower_sent)):
#                 bots_follower_sent.at[i, rel] = val_c.get(bots_follower_sent['id'][i], 0)
# rel2 = ['post', 'like', 'pinned']
# for rel in rel2:
#         rel_dt = pd.read_csv('edges/'+rel+'.csv')
#         rel_dt.set_index('source_id', inplace=True)
#         bots_follower_sent[rel] = ''
#         val_c = rel_dt.index.value_counts()
#         for i in range(len(bots_follower_sent)):
#                 bots_follower_sent.at[i, rel] = val_c.get(bots_follower_sent['id'][i], 0)

# # humans
# # count bots' relations
# humans_follower_sent = pd.DataFrame(humans_follower_sent)
# # followers
# rel_dt1, rel_dt2 = pd.read_csv('edges/followers.csv'), pd.read_csv('edges/following.csv')
# rel_dt1.set_index('source_id', inplace=True)
# rel_dt2.set_index('target_id', inplace=True)
# humans_follower_sent['followers'] = ''
# val_c1, val_c2 = rel_dt1.index.value_counts(), rel_dt2.index.value_counts()
# for i in range(len(humans_follower_sent)):
#         humans_follower_sent.at[i, 'followers'] = val_c1.get(humans_follower_sent['id'][i], 0) + val_c2.get(humans_follower_sent['id'][i], 0)
# # following
# rel_dt1, rel_dt2 = pd.read_csv('edges/following.csv'), pd.read_csv('edges/followers.csv')
# rel_dt1.set_index('source_id', inplace=True)
# rel_dt2.set_index('target_id', inplace=True)
# humans_follower_sent['following'] = ''
# val_c1, val_c2 = rel_dt1.index.value_counts(), rel_dt2.index.value_counts()
# for i in range(len(humans_follower_sent)):
#         humans_follower_sent.at[i, 'following'] = val_c1.get(humans_follower_sent['id'][i], 0) + val_c2.get(humans_follower_sent['id'][i], 0)
# # membership, mentioned
# rel1 = ['membership', 'mentioned']
# for rel in rel1:
#         rel_dt = pd.read_csv('edges/'+rel+'.csv')
#         rel_dt.set_index('target_id', inplace=True)
#         humans_follower_sent[rel] = ''
#         val_c = rel_dt.index.value_counts()
#         for i in range(len(humans_follower_sent)):
#                 humans_follower_sent.at[i, rel] = val_c.get(humans_follower_sent['id'][i], 0)
# rel2 = ['post', 'like', 'pinned']
# for rel in rel2:
#         rel_dt = pd.read_csv('edges/'+rel+'.csv')
#         rel_dt.set_index('source_id', inplace=True)
#         humans_follower_sent[rel] = ''
#         val_c = rel_dt.index.value_counts()
#         for i in range(len(humans_follower_sent)):
#                 humans_follower_sent.at[i, rel] = val_c.get(humans_follower_sent['id'][i], 0)

# bots_follower_sent.to_csv('t1_pos_botInfluencer.csv', index=False)
# bots_follower_sent.to_csv('t2_pos_botInfluencer.csv', index=False)
# bots_follower_sent.to_csv('t3_pos_botInfluencer.csv', index=False)
# bots_follower_sent.to_csv('t1_neg_botInfluencer.csv', index=False)
# bots_follower_sent.to_csv('t2_neg_botInfluencer.csv', index=False)
# bots_follower_sent.to_csv('t3_neg_botInfluencer.csv', index=False)

# humans_follower_sent.to_csv('t1_pos_humanInfluencer.csv', index=False)
# humans_follower_sent.to_csv('t2_pos_humanInfluencer.csv', index=False)
# humans_follower_sent.to_csv('t3_pos_humanInfluencer.csv', index=False)
# humans_follower_sent.to_csv('t1_neg_humanInfluencer.csv', index=False)
# humans_follower_sent.to_csv('t2_neg_humanInfluencer.csv', index=False)
# humans_follower_sent.to_csv('t3_neg_humanInfluencer.csv', index=False)
    
